# Merge Data - Classements Risques

<div class="alert alert-success">
    <b>Catégorisation des risques liées aux substances</b><br>Grâce à 3 fichiers distincts : Agritox,Substances préoccupantes (EU) et Reportable Ingredients List (Californie)<br>Nous allons catégoriser les risques grâce aux informations des trois fichiers, et faire une table qui catégorise chaque substance CAS/ EC.
</div>

In [1]:
# Importation des modules
import pandas as pd
import numpy as np

In [3]:
# Importation de la donnée Reportable List Ingredients
ril = pd.read_excel("/Users/alelia/Desktop/Data/DATA SUITS/Soutenance/Skincare/DATA/CLEAN/1. Chemicals in cosmetics/Reportable Ingredients List/RIL.xlsx")

In [82]:
# Importation de la donnée Substances Préoccupantes
substances_p = pd.read_excel("/Users/alelia/Desktop/Data/DATA SUITS/Soutenance/Skincare/DATA/CLEAN/1. Chemicals in cosmetics/Inventaire CE/candidate-list-of-svhc-for-authorisation-export.xlsx")

In [37]:
# Importation de la donnée Classement Danger, 
agritox = pd.read_csv("/Users/alelia/Desktop/Data/DATA SUITS/Soutenance/Skincare/DATA/CLEAN/2. AGRITOX/1. CLASSEMENT.csv", encoding = "ISO-8859-1", sep=';')

Les catégories uniques seront :
- Toxicité Cutanée et Respiratoire
- Risques Génétiques et Cancers
- Toxicité Systémique et Organique
- Toxicité Reproductive et Développementale
- Impact Environnemental et Accumulation


## AGRITOX

In [26]:
agritox.head()

,Numero CAS,NOM SA,CLASSEMENT DATE,CLASSEMENT CATÉGORIE DANGER,CLASSEMENT CODE H,CLASSEMENT MENTION DANGER
0,71751-41-2,abamectine,10/07/2012,Dangers pour le milieu aquatique - Danger aigu...,H400,Très toxique pour les organismes aquatiques
1,71751-41-2,abamectine,10/07/2012,Toxicité spécifique pour certains organes cibl...,H372,Risque avéré d'effet graves pour les organes à...
2,71751-41-2,abamectine,10/07/2012,"Toxicité pour la reproduction, catégorie 2(d)",H361d,Susceptible de nuire au foetus
3,71751-41-2,abamectine,10/07/2012,"Toxicité aiguë (par voie orale), catégorie 2",H300,Mortel en cas d'ingestion
4,71751-41-2,abamectine,10/07/2012,Dangers pour le milieu aquatique - Danger chro...,H410,"Très toxique pour les organismes aquatiques, e..."


In [ ]:
# Je transforme le type de donnes des CAS en string pour mener à bien le reste.

In [38]:
agritox['Numero CAS'] = agritox['Numero CAS'].astype(str)

In [140]:
# Pour les lignes avec plusieurs CAS, je m'assure de ne laisser qu'un CAS par ligne.

# Liste vide pour collecter les nouvelles lignes
new_rows = []

for index, row in agritox.iterrows():
    if " ou " in row["CAS"]:
        # Séparer le contenu de la colonne "CAS #" en deux parties
        cas = row["CAS"].split(' ou ')
        
        # Mettre à jour la première partie dans la ligne actuelle
        agritox.at[index, "CAS"] = cas[0]
        
        # Créer une nouvelle ligne avec la deuxième partie du CAS
        new_row = row.copy()  # Crée une copie de la ligne actuelle
        new_row["CAS"] = cas[1]  # Remplacer "CAS #" par la deuxième partie

        # Ajouter cette nouvelle ligne à la liste new_rows
        new_rows.append(new_row)

# Si des nouvelles lignes ont été collectées, les ajouter au DataFrame
if new_rows:
    # Créer un DataFrame à partir des nouvelles lignes et les concaténer
    agritox = pd.concat([agritox, pd.DataFrame(new_rows)], ignore_index=True)


In [40]:
# Observation des dangers uniques (pour savoir dans quelles catégories les mettre)
agritox["CLASSEMENT CATÉGORIE DANGER"].unique()

array(['Dangers pour le milieu aquatique - Danger aigu, catégorie 1',
       'Toxicité spécifique pour certains organes cibles - Exposition répétée, catégorie 1',
       'Toxicité pour la reproduction, catégorie 2(d)',
       'Toxicité aiguë (par voie orale), catégorie 2',
       'Dangers pour le milieu aquatique - Danger chronique, catégorie 1',
       'Toxicité aiguë (par inhalation), catégorie 1',
       'Sensibilisation cutanée, catégorie 1',
       'Toxicité spécifique pour certains organes cibles - Exposition répétée, catégorie 2',
       'Toxicité spécifique pour certains organes cibles - Exposition unique, catégorie 1',
       'Toxicité aiguë (par voie orale), catégorie 3',
       'Dangers pour le milieu aquatique - Danger chronique, catégorie 3',
       'Irritation oculaire, catégorie 2',
       'Irritant pour la peau, catégorie 2',
       'Toxicité spécifique pour certains organes cibles - Exposition unique - Irritation des voies respiratoires, catégorie 3',
       'Liquides 

In [41]:
# Création de la colonne Toxicité Cutanée et Respiratoire
agritox["Toxicité Cutanée et Respiratoire"] = [
    1 if any(keyword in row for keyword in ["peau", "respiratoire", "cutanée", "oculaire"]) 
    else 0
    for row in agritox["CLASSEMENT CATÉGORIE DANGER"]
]

In [43]:
# Création de la colonne Toxicité Systémique et Organique
agritox["Toxicité Systémique et Organique"] = [
    1 if any(keyword in row for keyword in ["Danger par aspiration", 
                                            "Toxicité aiguë (par inhalation)", 
                                            "Toxicité aiguë (par voie orale)", 
                                            "certains organes cibles",
                                            "narcotiques"]) 
    else 0
    for row in agritox["CLASSEMENT CATÉGORIE DANGER"]
]

In [44]:
# Création de la colonne Toxicité Reproductive et Développementale
agritox["Toxicité Reproductive et Développementale"] = [
    1 if any(keyword in row for keyword in ["reproduction", 
                                            "lactation"]) 
    else 0
    for row in agritox["CLASSEMENT CATÉGORIE DANGER"]
]

In [45]:
# Création de la colonne Risques Génétiques et Cancers
agritox["Toxicité Risques Génétiques et Cancers"] = [
    1 if any(keyword in row for keyword in ["Cancérogénicité"]) 
    else 0
    for row in agritox["CLASSEMENT CATÉGORIE DANGER"]
]

In [46]:
# Création de la colonne Impact Environnemental et Accumulation
agritox["Impact Environnemental et Accumulation"] = [
    1 if any(keyword in row for keyword in ["inflammables",
                                            "au contact de l'eau",
                                            "milieu aquatiqu"]) 
    else 0
    for row in agritox["CLASSEMENT CATÉGORIE DANGER"]
]

In [52]:
# Créer une colonne catégorie de Danger

new_column = []
for row in agritox["CLASSEMENT CATÉGORIE DANGER"]:
    try:
        # Séparer la chaîne de caractères par la virgule et prendre le deuxième élément
        categorie = row.split(",")[1].strip()  # .strip() pour enlever les espaces superflus
    except IndexError:
        # Si la séparation échoue (ex: il n'y a pas de virgule), ajouter un message par défaut
        categorie = "Pas de Catégorie"
    new_column.append(categorie)

# Ajouter la nouvelle colonne au DataFrame
agritox["Catégorie Danger"] = new_column

In [53]:
agritox.head()

,Numero CAS,NOM SA,CLASSEMENT DATE,CLASSEMENT CATÉGORIE DANGER,CLASSEMENT CODE H,CLASSEMENT MENTION DANGER,Toxicité Cutanée et Respiratoire,Toxicité Systémique et Organique,Toxicité Reproductive et Développementale,Toxicité Risques Génétiques et Cancers,Impact Environnemental et Accumulation,Catégorie Danger
0,71751-41-2,abamectine,10/07/2012,Dangers pour le milieu aquatique - Danger aigu...,H400,Très toxique pour les organismes aquatiques,0,0,0,0,1,catégorie 1
1,71751-41-2,abamectine,10/07/2012,Toxicité spécifique pour certains organes cibl...,H372,Risque avéré d'effet graves pour les organes à...,0,1,0,0,0,catégorie 1
2,71751-41-2,abamectine,10/07/2012,"Toxicité pour la reproduction, catégorie 2(d)",H361d,Susceptible de nuire au foetus,0,0,1,0,0,catégorie 2(d)
3,71751-41-2,abamectine,10/07/2012,"Toxicité aiguë (par voie orale), catégorie 2",H300,Mortel en cas d'ingestion,0,1,0,0,0,catégorie 2
4,71751-41-2,abamectine,10/07/2012,Dangers pour le milieu aquatique - Danger chro...,H410,"Très toxique pour les organismes aquatiques, e...",0,0,0,0,1,catégorie 1


In [54]:
agritox.columns

Index(['Numero CAS', 'NOM SA', 'CLASSEMENT DATE',
       'CLASSEMENT CATÉGORIE DANGER', 'CLASSEMENT CODE H',
       'CLASSEMENT MENTION DANGER', 'Toxicité Cutanée et Respiratoire',
       'Toxicité Systémique et Organique',
       'Toxicité Reproductive et Développementale',
       'Toxicité Risques Génétiques et Cancers',
       'Impact Environnemental et Accumulation', 'Catégorie Danger'],
      dtype='object')

In [55]:
# Changer l'ordre des colonnes
agritox = agritox[['Numero CAS', 'NOM SA', 'CLASSEMENT DATE', 'Toxicité Cutanée et Respiratoire',
       'Toxicité Systémique et Organique',
       'Toxicité Reproductive et Développementale',
       'Toxicité Risques Génétiques et Cancers',
       'Impact Environnemental et Accumulation', 'Catégorie Danger']]

In [102]:
# Dans agritox, les substances toxiques sont répertoriées, mais parfois, aucune toxicité ne lui ai attribuée. 
# Création d'une colonne Toxicité indéfinie

agritox["Toxicité indéfinie"] = [
    1 if (row["Toxicité Cutanée et Respiratoire"] + 
          row["Toxicité Systémique et Organique"] +
          row["Toxicité Reproductive et Développementale"] + 
          row["Toxicité Risques Génétiques et Cancers"] + 
          row["Impact Environnemental et Accumulation"]) == 0 
    else 0
    for index, row in agritox.iterrows()
]

In [114]:
agritox = agritox.rename(columns={'Numero CAS': 'CAS'})

In [139]:
agritox = agritox.rename(columns={'Toxicité Risques Génétiques et Cancers': 'Risques Génétiques et Cancers'})

In [106]:
agritox.columns

Index(['Numero CAS', 'NOM SA', 'CLASSEMENT DATE',
       'Toxicité Cutanée et Respiratoire', 'Toxicité Systémique et Organique',
       'Toxicité Reproductive et Développementale',
       'Risques Génétiques et Cancers',
       'Impact Environnemental et Accumulation', 'Catégorie Danger',
       'Toxicité indéfinie'],
      dtype='object')

In [107]:
agritox = agritox[['CAS', 'NOM SA', 'CLASSEMENT DATE',
       'Toxicité Cutanée et Respiratoire', 'Toxicité Systémique et Organique',
       'Toxicité Reproductive et Développementale',
       'Risques Génétiques et Cancers',
       'Impact Environnemental et Accumulation', 'Toxicité indéfinie','Catégorie Danger']]

In [141]:
# Enregistrement des données
agritox.to_excel("/Users/alelia/Desktop/Data/DATA SUITS/Soutenance/Skincare/DATA/CLEAN/2. AGRITOX/clean/agritox.xlsx")

In [115]:
agritox.head()

,CAS,NOM SA,CLASSEMENT DATE,Toxicité Cutanée et Respiratoire,Toxicité Systémique et Organique,Toxicité Reproductive et Développementale,Risques Génétiques et Cancers,Impact Environnemental et Accumulation,Toxicité indéfinie,Catégorie Danger
0,71751-41-2,abamectine,10/07/2012,0,0,0,0,1,0,catégorie 1
1,71751-41-2,abamectine,10/07/2012,0,1,0,0,0,0,catégorie 1
2,71751-41-2,abamectine,10/07/2012,0,0,1,0,0,0,catégorie 2(d)
3,71751-41-2,abamectine,10/07/2012,0,1,0,0,0,0,catégorie 2
4,71751-41-2,abamectine,10/07/2012,0,0,0,0,1,0,catégorie 1


## RIL

In [58]:
ril.columns

Index(['Unnamed: 0', 'Chemical Name', 'CAS RN', 'Synonyms', 'Hazard Traits',
       'When to Report (Any, FF, FA)'],
      dtype='object')

In [59]:
# Changer l'ordre des colonnes
ril = ril[['Chemical Name', 'CAS RN', 'Synonyms', 'Hazard Traits']]

In [60]:
ril.head()

,Chemical Name,CAS RN,Synonyms,Hazard Traits
0,Diquat dibromide,85-00-7,NaN,Toxicity Undefined
1,1-Chloro-4-nitrobenzene,100-00-5,4-Chloronitrobenzene,Carcinogenicity
2,4-Nitrophenol,100-02-7,NaN,Dermatotoxicity; Environmental tox; Hazard Tra...
3,para-Nitroanisole,100-17-4,NaN,Carcinogenicity
4,p-Dinitrobenzene,100-25-4,NaN,Reproductive Toxicity


In [65]:
# Création de la colonne Toxicité Cutanée et Respiratoire
ril.loc[:, "Toxicité Cutanée et Respiratoire"] = [
    1 if any(keyword in row for keyword in ["Dermatotoxicity", "Fragrance", 
                                            "Respiratory", "Ocular"]) 
    else 0
    for row in ril['Hazard Traits']
]

In [67]:
# Création de la colonne Toxicité Systémique et Organique
ril.loc[:, "Toxicité Systémique et Organique"] = [
    1 if any(keyword in row for keyword in ["Neurotoxi", 
                                            "Hepatotoxicity", 
                                            "Nephrotoxicity", 
                                            "Cardiovascular",
                                            "Musculoskeletal"]) 
    else 0
    for row in ril['Hazard Traits']
]

In [68]:
# Création de la colonne Toxicité Reproductive et Développementale
ril.loc[:, "Toxicité Reproductive et Développementale"] = [
    1 if any(keyword in row for keyword in ["Reproductive", 
                                            "Developmental", 
                                            "Endocrine"]) 
    else 0
    for row in ril['Hazard Traits']
]

In [69]:
# Création de la colonne Risques Génétiques et Cancers
ril.loc[:, "Risques Génétiques et Cancers"] = [
    1 if any(keyword in row for keyword in ["Carcinogenicity", 
                                            "Genotoxicity"]) 
    else 0
    for row in ril['Hazard Traits']
]

In [70]:
# Création de la colonne Impact Environnemental et Accumulation
ril["Impact Environnemental et Accumulation"] = [
    1 if any(keyword in row for keyword in ["Environmental",
                                            "Bioaccumulation"]) 
    else 0
    for row in ril['Hazard Traits']
]

In [73]:
ril = ril[['Chemical Name', 'CAS RN',
       'Toxicité Cutanée et Respiratoire', 'Toxicité Systémique et Organique',
       'Toxicité Reproductive et Développementale',
       'Risques Génétiques et Cancers',
       'Impact Environnemental et Accumulation']]

In [99]:
ril["Toxicité indéfinie"] = [
    1 if (row["Toxicité Cutanée et Respiratoire"] + 
          row["Toxicité Systémique et Organique"] +
          row["Toxicité Reproductive et Développementale"] + 
          row["Risques Génétiques et Cancers"] + 
          row["Impact Environnemental et Accumulation"]) == 0 
    else 0
    for index, row in ril.iterrows()
]

In [118]:
ril = ril.rename(columns={'CAS RN': 'CAS'})

In [111]:
ril.to_excel("/Users/alelia/Desktop/Data/DATA SUITS/Soutenance/Skincare/DATA/CLEAN/1. Chemicals in cosmetics/Reportable Ingredients List/clean/RIL.xlsx")

In [119]:
ril.head()

,Chemical Name,CAS,Toxicité Cutanée et Respiratoire,Toxicité Systémique et Organique,Toxicité Reproductive et Développementale,Risques Génétiques et Cancers,Impact Environnemental et Accumulation,Toxicité indéfinie
0,Diquat dibromide,85-00-7,0,0,0,0,0,1
1,1-Chloro-4-nitrobenzene,100-00-5,0,0,0,1,0,0
2,4-Nitrophenol,100-02-7,1,0,0,0,1,0
3,para-Nitroanisole,100-17-4,0,0,0,1,0,0
4,p-Dinitrobenzene,100-25-4,0,0,1,0,0,0


## Substances Préocupantes

In [83]:
substances_p.columns

Index(['Nom de la substance ', 'Description', 'CE', 'CAS',
       'Motif de l’inclusion ', 'Date d’inclusion ', 'Décision ',
       'Ensemble de données IUCLID ', 'Document d’appui ',
       'Réponses aux commentaires. ', 'Remarques '],
      dtype='object')

In [84]:
substances_p.head()

,Nom de la substance,Description,CE,CAS,Motif de l’inclusion,Date d’inclusion,Décision,Ensemble de données IUCLID,Document d’appui,Réponses aux commentaires.,Remarques
0,"Bis(α,α-dimethylbenzyl) peroxide",NaN,201-279-3,80-43-3,Toxic for reproduction (Article 57c),27-juin-2024,https://echa.europa.eu/documents/10162/733088c...,https://echa.europa.eu/documents/10162/d6f3d76...,https://echa.europa.eu/documents/10162/236dc0e...,https://echa.europa.eu/documents/10162/48e7c30...,NaN
1,Oligomerisation and alkylation reaction produc...,NaN,700-960-7,-,vPvB (Article 57e),23-janv.-2024,https://echa.europa.eu/documents/10162/0a8c79d...,https://echa.europa.eu/documents/10162/634c637...,https://echa.europa.eu/documents/10162/9a9d530...,https://echa.europa.eu/documents/10162/7cd990a...,"The name phenol, methylstyrenated and EC numbe..."
2,"Phenol, methylstyrenated",NaN,270-966-8,68512-30-1,vPvB (Article 57e),23-janv.-2024,https://echa.europa.eu/documents/10162/0a8c79d...,https://echa.europa.eu/documents/10162/634c637...,https://echa.europa.eu/documents/10162/9a9d530...,https://echa.europa.eu/documents/10162/7cd990a...,"The name phenol, methylstyrenated and EC numbe..."
3,Bumetrizole (UV-326),NaN,223-445-4,3896-11-5,vPvB (Article 57e),23-janv.-2024,https://echa.europa.eu/documents/10162/05ea7d0...,https://echa.europa.eu/documents/10162/c5b482b...,https://echa.europa.eu/documents/10162/94233c7...,https://echa.europa.eu/documents/10162/abe497c...,NaN
4,2-(dimethylamino)-2-[(4-methylphenyl)methyl]-1...,NaN,438-340-0,119344-86-4,Toxic for reproduction (Article 57c),23-janv.-2024,https://echa.europa.eu/documents/10162/fd512d8...,https://echa.europa.eu/documents/10162/035bf1c...,https://echa.europa.eu/documents/10162/0540648...,https://echa.europa.eu/documents/10162/1221ef8...,NaN


In [85]:
# Changer l'ordre des colonnes
substances_p = substances_p[['Nom de la substance ', 'CE', 'CAS',
       'Motif de l’inclusion ', 'Date d’inclusion ']]

In [89]:
# Création de la colonne Toxicité Cutanée et Respiratoire
substances_p.loc[:, "Toxicité Cutanée et Respiratoire"] = [
    1 if any(keyword in row for keyword in ["Respiratory"]) 
    else 0
    for row in substances_p['Motif de l’inclusion ']
]

In [90]:
# Création de la colonne Toxicité Systémique et Organique
substances_p.loc[:, "Toxicité Systémique et Organique"] = [
    1 if any(keyword in row for keyword in ["Equivalent level of concern having probable serious effects to human health", 
                                            "Specific target organ toxicity"]) 
    else 0
    for row in substances_p['Motif de l’inclusion ']
]

In [91]:
# Création de la colonne Toxicité Reproductive et Développementale
substances_p.loc[:, "Toxicité Reproductive et Développementale"] = [
    1 if any(keyword in row for keyword in ["reproduction", 
                                            "Endocrine"]) 
    else 0
    for row in substances_p['Motif de l’inclusion ']
]

In [92]:
# Création de la colonne Risques Génétiques et Cancers
substances_p.loc[:, "Risques Génétiques et Cancers"] = [
    1 if any(keyword in row for keyword in ["Carcinogenic", 
                                            "Mutagenic"]) 
    else 0
    for row in substances_p['Motif de l’inclusion ']
]

In [93]:
# Création de la colonne Impact Environnemental et Accumulation
substances_p["Impact Environnemental et Accumulation"] = [
    1 if any(keyword in row for keyword in ["PBT",
                                            "environment",
                                            "vPvB",
                                            "repeated exposure"]) 
    else 0
    for row in substances_p['Motif de l’inclusion ']
]

In [96]:
substances_p["Toxicité indéfinie"] = [
    1 if (row["Toxicité Cutanée et Respiratoire"] + 
          row["Toxicité Systémique et Organique"] +
          row["Toxicité Reproductive et Développementale"] + 
          row["Risques Génétiques et Cancers"] + 
          row["Impact Environnemental et Accumulation"]) == 0 
    else 0
    for index, row in substances_p.iterrows()
]

In [133]:
# Pareil que pour Agritox, il y a des lignes contenant plusieurs CAS, faire le même nettoyage

# Assurer que new_rows est une liste vide pour collecter les nouvelles lignes
new_rows = []

for index, row in substances_p.iterrows():
    if ", " in row["CAS"]:
        # Séparer le contenu de la colonne "CAS #" en deux parties
        cas = row["CAS"].split(', ')
        
        # Mettre à jour la première partie dans la ligne actuelle
        substances_p.at[index, "CAS"] = cas[0]
        
        # Créer une nouvelle ligne avec la ou les autres partie(s) du CAS
        for i in range(len(cas)-1):
            new_row = row.copy()  # Crée une copie de la ligne actuelle
            new_row["CAS"] = cas[i+1]  # Remplacer "CAS #" par la deuxième partie

            # Ajouter cette nouvelle ligne à la liste new_rows
            new_rows.append(new_row)

# Si des nouvelles lignes ont été collectées, les ajouter au DataFrame
if new_rows:
    # Créer un DataFrame à partir des nouvelles lignes et les concaténer
    substances_p = pd.concat([substances_p, pd.DataFrame(new_rows)], ignore_index=True)

In [135]:
# Enregistrement des données
substances_p.to_excel("/Users/alelia/Desktop/Data/DATA SUITS/Soutenance/Skincare/DATA/CLEAN/1. Chemicals in cosmetics/Inventaire CE/clean/substances_preoccupantes.xlsx")

In [109]:
substances_p.head()

,Nom de la substance,CE,CAS,Motif de l’inclusion,Date d’inclusion,Toxicité Cutanée et Respiratoire,Toxicité Systémique et Organique,Toxicité Reproductive et Développementale,Risques Génétiques et Cancers,Impact Environnemental et Accumulation,Toxicité indéfinie
0,"Bis(α,α-dimethylbenzyl) peroxide",201-279-3,80-43-3,Toxic for reproduction (Article 57c),27-juin-2024,0,0,1,0,0,0
1,Oligomerisation and alkylation reaction produc...,700-960-7,-,vPvB (Article 57e),23-janv.-2024,0,0,0,0,1,0
2,"Phenol, methylstyrenated",270-966-8,68512-30-1,vPvB (Article 57e),23-janv.-2024,0,0,0,0,1,0
3,Bumetrizole (UV-326),223-445-4,3896-11-5,vPvB (Article 57e),23-janv.-2024,0,0,0,0,1,0
4,2-(dimethylamino)-2-[(4-methylphenyl)methyl]-1...,438-340-0,119344-86-4,Toxic for reproduction (Article 57c),23-janv.-2024,0,0,1,0,0,0


## MERGE

Le but est de créer une et même table contenant toutes les informations des 3 tables.
Parfois, pour un CAS, un danger sera présent dans une table et non dans une autre. 
Je décide donc que le danger dans une table l'emporte sur la non présence de danger sur une autre table.

In [147]:
mergedata = pd.concat([substances_p, ril, agritox], axis=0, ignore_index=True)

In [148]:
# Appliquer des fonctions d'agrégation spécifiques à chaque colonne
mergedata = mergedata.groupby('CAS', as_index=False).agg({
    'Toxicité Cutanée et Respiratoire': 'max',  # Prendre le max pour les colonnes booléennes
    'Toxicité Systémique et Organique': 'max',
    'Toxicité Reproductive et Développementale': 'max',
    'Risques Génétiques et Cancers': 'max',
    'Impact Environnemental et Accumulation': 'max',
    'CE' : 'first'
})

In [151]:
mergedata["Toxicité indéfinie"] = [
    1 if (row["Toxicité Cutanée et Respiratoire"] + 
          row["Toxicité Systémique et Organique"] +
          row["Toxicité Reproductive et Développementale"] + 
          row["Risques Génétiques et Cancers"] + 
          row["Impact Environnemental et Accumulation"]) == 0 
    else 0
    for index, row in mergedata.iterrows()
]

In [153]:
mergedata.columns

Index(['CAS', 'Toxicité Cutanée et Respiratoire',
       'Toxicité Systémique et Organique',
       'Toxicité Reproductive et Développementale',
       'Risques Génétiques et Cancers',
       'Impact Environnemental et Accumulation', 'CE', 'Toxicité indéfinie'],
      dtype='object')

In [154]:
mergedata = mergedata[['CAS', 'Toxicité Cutanée et Respiratoire',
       'Toxicité Systémique et Organique',
       'Toxicité Reproductive et Développementale',
       'Risques Génétiques et Cancers',
       'Impact Environnemental et Accumulation', 'Toxicité indéfinie', 'CE']]

In [155]:
mergedata.to_excel("/Users/alelia/Desktop/Data/DATA SUITS/Soutenance/Skincare/DATA/CLEAN/Substances Préoccupantes/merge.xlsx")